In [1]:
import numpy as np
import plotly.graph_objects as go

In [2]:
class edgecnt:
    PE, CE, OE, EE, ME = 0, 0, 0, [0, 0, 0, 0], [0, 0, 0, 0]
    
ppos_req = ["overall", "noun", "verb", "properNoun", "adjective", "adverb", "determiner", "numeral", "pronoun", "preposition"]

def getRaw(lang):
    metrics = {}
    print("Language: " + lang)
    path = "Results/Expts/RDF-Outer-Use10Gen1/Analysis/" + lang + "/POS_RAW.txt"
    inp_file = open(path, "r")
    inp = inp_file.read().splitlines()
    it = 0
    
    while(it<len(inp)):
        temp = edgecnt()
        pos = inp[it]
        temp.OE = int(inp[it+1])
        temp.CE = int(inp[it+2])
        temp.EE = inp[it+3].split()
        temp.ME = inp[it+4].split()
        it+=5
        
        for i in range(4):
            temp.EE[i] = int(temp.EE[i])
            temp.ME[i] = int(temp.ME[i])
        temp.PE = sum(temp.EE)
        temp.PE += temp.CE
        metrics.update({pos : temp})
        
    return metrics
    
def compute(metrics):
    bvP, bvR, ipnoco, ionpci = [], [], [], []
    for i in range(5):
        ipnoco.append([])
        ionpci.append([])
        
    for pos in ppos_req:
        m = metrics[pos]
        bvp = m.CE * 100.00/ max(1, (m.CE + m.EE[3]))
        bvP.append(bvp)
        bvr = m.CE * 100.00 / max(1, (m.CE + m.ME[3]))
        bvR.append(bvr)
        
        sumipnoco, sumionpci = 0, 0
        for i in range(1, 5):
            sumipnoco += m.EE[i-1]*100.00/max(1, m.PE)
            sumionpci += m.ME[i-1]*100.00/max(1, m.OE)
            ipnoco[i].append(m.EE[i-1]*100.00/max(1, m.PE))
            ionpci[i].append(m.ME[i-1]*100.00/max(1, m.OE))
        ipnoco[0].append(100 - sumipnoco)
        ionpci[0].append(100 - sumionpci)
        
    return bvP, bvR, ipnoco, ionpci

def dataprinter(bvP, bvR, ipnoco, ionpci):
    i = 0
    for pos in ppos_req:
        print("Part of Speech: " + pos)
        print("Precision when both vertices common in Predicted and Original: " + str(bvP[i]))
        print("Recall when both vertices commin in Original and Data: " + str(bvR[i]) + "\n")
        print("Common vertices legend: 0 = No vertex, 1 = lang1, 2 = lang2, 3 = both vertices")
        print("In Predicted, Not in Original, Classified by Original, as percentage of Predicted (Precision')")
        for j in range(1, 5):
            print(str(j-1) + " - " + str(ipnoco[j][i]))
        print("In Original, Not in Predicted, Classified by Input, as percentage of Original (Recall)")
        for j in range(1, 5):
            print(str(j-1) + " - " + str(ionpci[j][i]))
        
        print("")
        i+=1

In [3]:
def printdatacount(metrics):
    for pos in ppos_req:
        ratio = metrics[pos].PE*100/metrics[pos].OE
        print(pos + " - Translations in Apertium Bidix: " + str(metrics[pos].OE) + 
                  ", Percentage of Translations Predicted: %.2f%%" % ratio
             )
    
def prefixSum(arr):
    ps = []
    for i in range(len(arr)):
        ps.append([])
        for j in range(len(arr[i])):
            ps[i].append(arr[i][j])
            if(i>0):
                ps[i][j] += ps[i-1][j]
    return ps

def plot4in1(bvP, bvR, ipnoco, ionpci):
    subcat = ["Neither", "Left", "Right", "Both"]
    fig = go.Figure()
    fig.add_trace(go.Bar(name="Both Vertex Precision", x=ppos_req, y=bvP, base=0, offset=-0.4, width=0.2))
    fig.add_trace(go.Bar(name="Both Vertex Recall", x=ppos_req, y=bvR, base=0, offset=-0.2, width=0.2))
    
    PSipnoco = prefixSum(ipnoco)
    PSionpci = prefixSum(ionpci)
    
    for i in range(4, 0, -1):
        name = subcat[i-1] + " with Original"
        fig.add_trace(go.Bar(name=name, x=ppos_req, y=PSipnoco[i], base=0, offset=0, width=0.2))
    fig.add_trace(go.Bar(name="Overall Precision", x=ppos_req, y=PSipnoco[0], base=0, offset=0, width=0.2))
    
    for i in range(4, 0, -1):
        name = subcat[i-1] + " with Input"
        fig.add_trace(go.Bar(name=name, x=ppos_req, y=PSionpci[i], base=0, offset=0.2, width=0.2))
    fig.add_trace(go.Bar(name="Overall Recall", x=ppos_req, y=PSionpci[0], base=0, offset=0.2, width=0.2))
    
    
    fig.update_layout(barmode='group', width=1000, height=800, xaxis_tickangle=45)
    fig.show()
    
def plot2in2(bvP, bvR, ipnoco, ionpci):
    subcat = ["Neither", "Left", "Right", "Both"]
    fig = go.Figure()
    fig2 = go.Figure()
    fig.add_trace(go.Bar(name="Both Vertex Precision", x=ppos_req, y=bvP, base=0, offset=-0.4, width=0.4))
    fig.add_trace(go.Bar(name="Both Vertex Recall", x=ppos_req, y=bvR, base=0, offset=0, width=0.4))
    
    PSipnoco = prefixSum(ipnoco)
    PSionpci = prefixSum(ionpci)
    
    for i in range(4, 0, -1):
        name = subcat[i-1] + " with Original"
        fig2.add_trace(go.Bar(name=name, x=ppos_req, y=PSipnoco[i], base=0, offset=-0.4, width=0.4))
    fig2.add_trace(go.Bar(name="Overall Precision", x=ppos_req, y=PSipnoco[0], base=0, offset=-0.4, width=0.4))
    
    for i in range(4, 0, -1):
        name = subcat[i-1] + " with Input"
        fig2.add_trace(go.Bar(name=name, x=ppos_req, y=PSionpci[i], base=0, offset=0, width=0.4))
    fig2.add_trace(go.Bar(name="Overall Recall", x=ppos_req, y=PSionpci[0], base=0, offset=0, width=0.4))
    
    
    fig.update_layout(barmode='group', width=1000, height=800, xaxis_tickangle=45)
    fig.show()
    fig2.update_layout(barmode='group', width=1000, height=800, xaxis_tickangle=45)
    fig2.show()
    
def GetLang(langpair):
    metrics = getRaw(langpair)
    bvP, bvR, ipnoco, ionpci = compute(metrics)
    printdatacount(metrics)
    plot2in2(bvP, bvR, ipnoco, ionpci)

<h2> Hyperparameters Used </h2>
Context Depth - 3 <br>
Maximum Cycle Length - 7 <br>
Source Language Repetition - Not allowed <br>
Cutoff for being 'Large Context' - 5 adjacent vertices to source <br>
Minimum Cycle Length in Large Contexts - 5 <br>
Minimum Cycle Length in Small Contexts - 4 <br>
Confidence Multiplier if Target has degree > 2 - 1.4 <br>
Confidence Threshold for predictions - 0.7 <br>

The graphs have been plotted using plotly, which allows great interactivty: can hover over bars to see values, crop, zoom in, zoom out etc. (tools available on hovering a little above the top right of each graph).
<h2> Understanding the Graphs </h2>
Data is visualized for 10 categories. The first 'category' shows overall results. This is followed by 9 part of speech categories roughly in decreasing order of priority.<br> 
The first graph contains 2 bars for each category:
<ol>
   <li> The blue bar shows precision when both words of the predicted translation are in the apertium bilingual dictionary for the language pair. This partially resolves data insufficiency issues (language-pairs are not exhaustive with all translations). It is still possible that the actual precision is higher as the predicted translations are not wrong, just not present in the language-pair. In some sense, a minimum bound on precision is thus provided.
   <li> The red bar shows recall when both words of an original apertium translation are in the data used (10 other language-pairs) but have not been predicted as a translation. This partially resolves data-insufficiency issues (other language-pairs not having data to infer this translation). 
</ol>
The second graph contains 2 bars for each category, each bar split into 5 parts:
<ol>
    <li> The first bar compares Predicted translations to Original (Apertium bidix) translations. The lowest part (in orange) shows percentage of predicted translations that were in original data. This is annotated overall precision which may be misleading as Apertium data can be incomplete (refer to graph 1 for realistic numbers). The extra translations not in original data are further classified into 4 categories based on Original data. Note iPnOcO stands for In Predicted, Not in Original, Classified by Original.
    <ol>
        <li> Category iPnOcO 0: Neither of the words in the predicted translation are in Original data.
        <li> Category iPnOcO 1: Language 1 (english in "en-es" pair) word of translation is in Original data.
        <li> Category iPnOcO 2: Language 2 (spanish in "en-es" pair) word of translation is in Original data.
        <li> Category iPnOcO 3: Both words of the predicted translation are in Original Data.
    </ol>
    <br>
    <li> The second bar compares Original translations to Predicted translations. The lowest part (in yellow) shows percentage of original translations that were in the predicted data. This is annotated overall recall. The recall shown in Graph 1 is slightly different, in that it ignores translations which could not possibly have been inferred as either one or both words don't exist in Input data (other 10 language-pairs). The translations in Original that are not in Predicted are classified in 4 categories based on Input data. Note iOnPcI stands for In Original, Not in Predicted, Classified by Input.
    <ol>
        <li> Category iOnPcI 0: Neither of the words in the original translation are in input data.
        <li> Category iOnPcI 1: Language 1 (english in "en-es" pair) word of translation is in input data.
        <li> Category iOnPcI 2: Language 2 (spanish in "en-es" pair) word of translation is in input data.
        <li> Category iOnPcI 3: Both words of the predicted translation are in input data.
    </ol>
</ol>

<h4> Overall Results </h4>

In [9]:
langlist = ["eng-spa", "eng-cat", "fra-spa", "fra-cat", "epo-fra", "epo-cat", "epo-eng", "epo-spa", "oci-cat", "oci-spa", "oci-fra"]
avgpredratio = {}
for pos in ppos_req:
    avgpredratio.update({pos : 0})
abvP, abvR, aipnoco, aionpci = [0]*10, [0]*10, [ [0]*10 for _ in range(5)], [ [0]*10 for _ in range(5)]
for lang in langlist:
    metrics = getRaw(lang)
    for pos in ppos_req:
        avgpredratio[pos] += metrics[pos].PE*100/metrics[pos].OE
    bvP, bvR, ipnoco, ionpci = compute(metrics)
    for i in range(len(bvP)):
        abvP[i] += bvP[i]
        abvR[i] += bvR[i]
        for j in range(5):
            aipnoco[j][i] += ipnoco[j][i]
            aionpci[j][i] += ionpci[j][i]
    
for i in range(10):
    abvP[i]/=len(langlist)
    abvR[i]/=len(langlist)
    for j in range(5):
        aipnoco[j][i]/=len(langlist)
        aionpci[j][i]/=len(langlist)
for pos in avgpredratio:
    avgpredratio[pos]/=len(langlist)
    
for pos in ppos_req:
    print(pos + " - Percentage of Translations Predicted: %.2f%%" % avgpredratio[pos] )
plot2in2(abvP, abvR, aipnoco, aionpci)



Language: eng-spa
Language: eng-cat
Language: fra-spa
Language: fra-cat
Language: epo-fra
Language: epo-cat
Language: epo-eng
Language: epo-spa
Language: oci-cat
Language: oci-spa
Language: oci-fra
overall - Percentage of Translations Predicted: 72.23%
noun - Percentage of Translations Predicted: 49.99%
verb - Percentage of Translations Predicted: 51.04%
properNoun - Percentage of Translations Predicted: 491.96%
adjective - Percentage of Translations Predicted: 39.47%
adverb - Percentage of Translations Predicted: 38.54%
determiner - Percentage of Translations Predicted: 56.42%
numeral - Percentage of Translations Predicted: 208.76%
pronoun - Percentage of Translations Predicted: 90.21%
preposition - Percentage of Translations Predicted: 66.69%


<h4> Analysis </h4>
In the experiment, 11 language pairs are taken. Each language pair is removed and generated using the other 10. The results are averaged (divided by 11) across POS categories so that all language-pairs (large and small in data) are given equal weightage. The percentages presented here-on are averages. <br>
The prediction set size is 25.63% of Apertium bidix. This ratio is very high (78%) for pronouns, high (68%) for nouns and determiners and also good for properNouns (55%). Notice that the overall average is lower than all categories except numerals (20%).
    <summary> Is this an error? No. </summary>
    This is not because of some error, but rather because the overall is an average over language-pairs, which in turn are a weighted average of the categories. The category-wise %ages are instead direct averages across language-pairs. In a particular language-pair, it is intuitive that when the predicted percentage is high for a category, that category is probably going to have less translations originally (which is used for weighting). Thus the weighting for language-pair overall computation brings the overall average down.

<h5> Graph 1 - Both vertices common </h5>
Precision is fairly high at 88%. Particularly good for pronouns (99.5%) and numerals (97.8%). Low for pronouns (67%), prepositions (75%) and slightly for determiners and adverbs too (~81-82%). Note however that these 4 have the highest recalls (Determiners 62%, Prepositions 52%, Pronouns 43%, Adverbs 38%). This is high when compared to an overall average of 24%. ProperNouns have particularly low recall (14%). This is a little concerning as both vertices are present, but also expected because many language pairs may not have the same set of properNouns leading to a lack of cycles.
<h5> Graph 2 - Overall classification </h5>
Overall precision is 60% and recall 13%. The significant drop in both compared to Graph 1 just shows the incomplete-ness of Apertium data. Notice that the differences between the 2 sub-categories of one-vertex prec/recall in this graph don't mean much because the choice of lang1 and lang2 was arbitrary for a pair, and trends comparing these 2 are purely co-incidental. 
<b> Left bar </b> (% wrt predicted data compared to apertium lang-pair (original) ): </b> <br>
Most 'precision-misses' are mostly because neither vertex is present (22%, 55% of extra predictions). One vertex is missing in 13% cases whereas both in only 8%. This is good. Neither vertex is particularly prevalent in Nouns (23%), ProperNouns (37%). Both vertices missing happen prevalently in adverbs (14%), determiners (17%), pronouns (22%), and prepositions (21%). 1-vertex-missing is the major cause of 'extra' predictions only in pronouns (27%) and perhaps significant in determiners and adverbs. <br>
<b> Right bar </b> (% wrt apertium lang-pair data (original) compared to input data): <br>
Overall recall-misses are least due to neither vertex being there in the original data (12%, 15% of misses). 1 word missing (35%) and both present (40%) are the significant categories. Overall recall has trends quite similar to recall as in Graph 1. Neither vertex misses are significant mainly in ProperNouns (16%) and adverbs (17%). Both vertex recall misses are roughly similar across categories (30%-50%), with verbs (50%) on the higher side and properNouns/prepositions (both 32%) on the lower. 1-vertex-recall is lower for pronouns (20%) and particularly high mainly for properNouns (46%) and numerals (37%). <br>
It may be worth-while to divide the both-vertex data into have-common-bicomp or not for further breakdown/analysis. This is because vertices that share a biconnected component have a cycle through both of them.

In [50]:
GetLang("eng-spa")

Language: eng-spa
overall - Translations in Apertium Bidix: 35429, Percentage of Translations Predicted: 62.71%
noun - Translations in Apertium Bidix: 28903, Percentage of Translations Predicted: 36.13%
verb - Translations in Apertium Bidix: 7088, Percentage of Translations Predicted: 63.85%
properNoun - Translations in Apertium Bidix: 12608, Percentage of Translations Predicted: 192.10%
adjective - Translations in Apertium Bidix: 9496, Percentage of Translations Predicted: 31.87%
adverb - Translations in Apertium Bidix: 4473, Percentage of Translations Predicted: 33.13%
determiner - Translations in Apertium Bidix: 212, Percentage of Translations Predicted: 24.53%
numeral - Translations in Apertium Bidix: 143, Percentage of Translations Predicted: 173.43%
pronoun - Translations in Apertium Bidix: 172, Percentage of Translations Predicted: 54.65%
preposition - Translations in Apertium Bidix: 372, Percentage of Translations Predicted: 62.90%


<h4> Analysis </h4>
The language-pair is medium sized with 32k edges. The prediction set size is 7.18% of Apertium bidix. This ratio is very very low for nouns (3.56%) and especially properNouns (0.35%).
<h5> Graph 1 - Both vertices common </h5>
The precision is fairly high, ~86%. Recall is however low at ~12%. As evident for all future language pairs, properNouns have near 100% precision. Numerals too. Here Verbs, Adjectives and Determiners have 90-95% Precision. As evident for all language-pairs, pronouns perform poorly. Recall is decent for the last 4 categories, but low earlier, lowest for properNouns (~3%). 

<h5> Graph 2 - Overall classification </h5>
<b> Left bar </b> (% wrt predicted data compared to apertium lang-pair (original) ): </b> <br>
The overall precision is very high in Determiners and Numerals. In properNouns, all extra predictions have neither word in the original data which shows great structure. 13% of predictions overall are extra but with both vertices present in original data. This number is particularly high for pronouns, prepositions, adverbs and nouns. The english equivalent of 6% verbs and 12% adjectives predicted translations) is not there in Apertium data. <br>
<b> Right bar </b> (% wrt apertium lang-pair data (original) compared to input data): <br>
Overall recall is low (5.3%), especially in properNouns(0.2%) and nouns (2.9%). It is higher in adverbs, determiners and especially prepositions (39%). Overall, 41% of original data translations have both words in input data, which is unfortunately high. In Verbs, it is as high as 59%. On the contrary, in 23% of original translations, neither word is there in input data. This is especially high in properNouns (36%), determiners (39%) and pronouns (34%). There are suggestions of Spanish input data being lower compared to this language pair for adverbs (37%), verbs (17%), numerals (58%) and determiners. On the other hand more English data is missing for properNouns (61%), nouns (20%).

In [51]:
GetLang("eng-cat")

Language: eng-cat
overall - Translations in Apertium Bidix: 52038, Percentage of Translations Predicted: 20.90%
noun - Translations in Apertium Bidix: 36821, Percentage of Translations Predicted: 25.22%
verb - Translations in Apertium Bidix: 7890, Percentage of Translations Predicted: 49.25%
properNoun - Translations in Apertium Bidix: 38517, Percentage of Translations Predicted: 10.12%
adjective - Translations in Apertium Bidix: 11573, Percentage of Translations Predicted: 21.39%
adverb - Translations in Apertium Bidix: 8254, Percentage of Translations Predicted: 17.03%
determiner - Translations in Apertium Bidix: 111, Percentage of Translations Predicted: 82.88%
numeral - Translations in Apertium Bidix: 141, Percentage of Translations Predicted: 185.82%
pronoun - Translations in Apertium Bidix: 194, Percentage of Translations Predicted: 42.27%
preposition - Translations in Apertium Bidix: 336, Percentage of Translations Predicted: 83.33%


<h4> Analysis </h4>
The language-pair is small sized with 13k edges. The prediction set size is (47%) of Apertium bidix (quite high ratio). It is especially high for nouns (483% of original size) which suggests the data has very little information on nouns. 
<h5> Graph 1 </h5>
The precision is fairly high, ~91%. Recall is however low at ~12%. As evident for all future language pairs, properNouns have near 100% precision. Numerals too. Here Verbs, Adjectives and Determiners have 90-95% Precision. As evident for all language-pairs, pronouns perform poorly. Recall is decent for the last 4 categories, but low earlier, lowest for properNouns (~3%). 

<h5> Graph 2 </h5>
<b> Left bar </b> (% wrt predicted data compared to apertium lang-pair (original) ): </b> <br>
The overall precision is very high in Determiners and Numerals. In properNouns, all extra predictions have neither word in the original data which shows great structure. 13% of predictions overall are extra but with both vertices present in original data. This number is particularly high for pronouns, prepositions, adverbs and nouns. The english equivalent of 6% verbs and 12% adjectives predicted translations) is not there in Apertium data. <br>
<b> Right bar </b> (% wrt apertium lang-pair data (original) compared to input data): <br>
Overall recall is low (5.3%), especially in properNouns(0.2%) and nouns (2.9%). It is higher in adverbs, determiners and especially prepositions (39%). Overall, 41% of original data translations have both words in input data, which is unfortunately high. In Verbs, it is as high as 59%. On the contrary, in 23% of original translations, neither word is there in input data. This is especially high in properNouns (36%), determiners (39%) and pronouns (34%). There are suggestions of Spanish input data being lower compared to this language pair for adverbs (37%), verbs (17%), numerals (58%) and determiners. On the other hand more English data is missing for properNouns (61%), nouns (20%).

In [15]:
GetLang("eng-spa")

Language: eng-spa
overall - Translations in Apertium Bidix: 35429, Percentage of Translations Predicted: 55.50%
noun - Translations in Apertium Bidix: 28903, Percentage of Translations Predicted: 27.38%
verb - Translations in Apertium Bidix: 7088, Percentage of Translations Predicted: 43.43%
properNoun - Translations in Apertium Bidix: 12608, Percentage of Translations Predicted: 192.10%
adjective - Translations in Apertium Bidix: 9496, Percentage of Translations Predicted: 25.11%
adverb - Translations in Apertium Bidix: 4473, Percentage of Translations Predicted: 24.50%
determiner - Translations in Apertium Bidix: 212, Percentage of Translations Predicted: 17.92%
numeral - Translations in Apertium Bidix: 143, Percentage of Translations Predicted: 173.43%
pronoun - Translations in Apertium Bidix: 172, Percentage of Translations Predicted: 39.53%
preposition - Translations in Apertium Bidix: 372, Percentage of Translations Predicted: 53.23%


In [16]:
GetLang("fra-cat")

Language: fra-cat
overall - Translations in Apertium Bidix: 67858, Percentage of Translations Predicted: 20.07%
noun - Translations in Apertium Bidix: 41553, Percentage of Translations Predicted: 23.70%
verb - Translations in Apertium Bidix: 10675, Percentage of Translations Predicted: 36.80%
properNoun - Translations in Apertium Bidix: 58104, Percentage of Translations Predicted: 15.27%
adjective - Translations in Apertium Bidix: 19550, Percentage of Translations Predicted: 13.40%
adverb - Translations in Apertium Bidix: 4522, Percentage of Translations Predicted: 23.57%
determiner - Translations in Apertium Bidix: 61, Percentage of Translations Predicted: 104.92%
numeral - Translations in Apertium Bidix: 229, Percentage of Translations Predicted: 182.53%
pronoun - Translations in Apertium Bidix: 203, Percentage of Translations Predicted: 49.26%
preposition - Translations in Apertium Bidix: 505, Percentage of Translations Predicted: 38.02%


In [17]:
GetLang("epo-fra")

Language: epo-fra
overall - Translations in Apertium Bidix: 42809, Percentage of Translations Predicted: 55.57%
noun - Translations in Apertium Bidix: 43265, Percentage of Translations Predicted: 23.96%
verb - Translations in Apertium Bidix: 16384, Percentage of Translations Predicted: 22.49%
properNoun - Translations in Apertium Bidix: 5935, Percentage of Translations Predicted: 493.24%
adjective - Translations in Apertium Bidix: 13228, Percentage of Translations Predicted: 22.10%
adverb - Translations in Apertium Bidix: 5205, Percentage of Translations Predicted: 15.98%
determiner - Translations in Apertium Bidix: 82, Percentage of Translations Predicted: 63.41%
numeral - Translations in Apertium Bidix: 404, Percentage of Translations Predicted: 34.65%
pronoun - Translations in Apertium Bidix: 181, Percentage of Translations Predicted: 32.04%
preposition - Translations in Apertium Bidix: 562, Percentage of Translations Predicted: 28.83%


In [18]:
GetLang("epo-cat")

Language: epo-cat
overall - Translations in Apertium Bidix: 40882, Percentage of Translations Predicted: 42.51%
noun - Translations in Apertium Bidix: 22813, Percentage of Translations Predicted: 63.96%
verb - Translations in Apertium Bidix: 6688, Percentage of Translations Predicted: 80.86%
properNoun - Translations in Apertium Bidix: 40198, Percentage of Translations Predicted: 21.34%
adjective - Translations in Apertium Bidix: 7012, Percentage of Translations Predicted: 58.50%
adverb - Translations in Apertium Bidix: 4345, Percentage of Translations Predicted: 28.77%
determiner - Translations in Apertium Bidix: 108, Percentage of Translations Predicted: 64.81%
numeral - Translations in Apertium Bidix: 66, Percentage of Translations Predicted: 600.00%
pronoun - Translations in Apertium Bidix: 74, Percentage of Translations Predicted: 121.62%
preposition - Translations in Apertium Bidix: 234, Percentage of Translations Predicted: 77.78%


In [19]:
GetLang("epo-eng")

Language: epo-eng
overall - Translations in Apertium Bidix: 32068, Percentage of Translations Predicted: 71.04%
noun - Translations in Apertium Bidix: 32604, Percentage of Translations Predicted: 38.64%
verb - Translations in Apertium Bidix: 12947, Percentage of Translations Predicted: 32.30%
properNoun - Translations in Apertium Bidix: 766, Percentage of Translations Predicted: 3073.63%
adjective - Translations in Apertium Bidix: 12534, Percentage of Translations Predicted: 26.15%
adverb - Translations in Apertium Bidix: 4378, Percentage of Translations Predicted: 32.94%
determiner - Translations in Apertium Bidix: 114, Percentage of Translations Predicted: 63.16%
numeral - Translations in Apertium Bidix: 266, Percentage of Translations Predicted: 34.59%
pronoun - Translations in Apertium Bidix: 123, Percentage of Translations Predicted: 73.17%
preposition - Translations in Apertium Bidix: 214, Percentage of Translations Predicted: 85.05%


In [20]:
GetLang("epo-spa")

Language: epo-spa
overall - Translations in Apertium Bidix: 47241, Percentage of Translations Predicted: 30.41%
noun - Translations in Apertium Bidix: 27892, Percentage of Translations Predicted: 41.37%
verb - Translations in Apertium Bidix: 7505, Percentage of Translations Predicted: 55.00%
properNoun - Translations in Apertium Bidix: 41825, Percentage of Translations Predicted: 19.68%
adjective - Translations in Apertium Bidix: 9913, Percentage of Translations Predicted: 31.15%
adverb - Translations in Apertium Bidix: 6172, Percentage of Translations Predicted: 14.91%
determiner - Translations in Apertium Bidix: 88, Percentage of Translations Predicted: 72.73%
numeral - Translations in Apertium Bidix: 98, Percentage of Translations Predicted: 381.63%
pronoun - Translations in Apertium Bidix: 135, Percentage of Translations Predicted: 65.19%
preposition - Translations in Apertium Bidix: 468, Percentage of Translations Predicted: 38.89%


In [21]:
GetLang("oci-cat")

Language: oci-cat
overall - Translations in Apertium Bidix: 18657, Percentage of Translations Predicted: 135.94%
noun - Translations in Apertium Bidix: 12838, Percentage of Translations Predicted: 56.18%
verb - Translations in Apertium Bidix: 5094, Percentage of Translations Predicted: 68.20%
properNoun - Translations in Apertium Bidix: 7637, Percentage of Translations Predicted: 479.74%
adjective - Translations in Apertium Bidix: 4728, Percentage of Translations Predicted: 34.14%
adverb - Translations in Apertium Bidix: 2344, Percentage of Translations Predicted: 35.67%
determiner - Translations in Apertium Bidix: 137, Percentage of Translations Predicted: 49.64%
numeral - Translations in Apertium Bidix: 403, Percentage of Translations Predicted: 115.14%
pronoun - Translations in Apertium Bidix: 237, Percentage of Translations Predicted: 23.63%
preposition - Translations in Apertium Bidix: 428, Percentage of Translations Predicted: 68.22%


In [22]:
GetLang("oci-spa")

Language: oci-spa
overall - Translations in Apertium Bidix: 15351, Percentage of Translations Predicted: 136.28%
noun - Translations in Apertium Bidix: 12770, Percentage of Translations Predicted: 50.78%
verb - Translations in Apertium Bidix: 4545, Percentage of Translations Predicted: 59.01%
properNoun - Translations in Apertium Bidix: 5480, Percentage of Translations Predicted: 536.39%
adjective - Translations in Apertium Bidix: 4220, Percentage of Translations Predicted: 34.98%
adverb - Translations in Apertium Bidix: 2176, Percentage of Translations Predicted: 35.48%
determiner - Translations in Apertium Bidix: 113, Percentage of Translations Predicted: 44.25%
numeral - Translations in Apertium Bidix: 457, Percentage of Translations Predicted: 115.10%
pronoun - Translations in Apertium Bidix: 236, Percentage of Translations Predicted: 38.14%
preposition - Translations in Apertium Bidix: 400, Percentage of Translations Predicted: 61.50%


In [23]:
GetLang("oci-fra")

Language: oci-fra
overall - Translations in Apertium Bidix: 40408, Percentage of Translations Predicted: 32.35%
noun - Translations in Apertium Bidix: 24772, Percentage of Translations Predicted: 36.61%
verb - Translations in Apertium Bidix: 6530, Percentage of Translations Predicted: 54.67%
properNoun - Translations in Apertium Bidix: 35670, Percentage of Translations Predicted: 21.35%
adjective - Translations in Apertium Bidix: 9726, Percentage of Translations Predicted: 29.20%
adverb - Translations in Apertium Bidix: 2327, Percentage of Translations Predicted: 65.06%
determiner - Translations in Apertium Bidix: 64, Percentage of Translations Predicted: 137.50%
numeral - Translations in Apertium Bidix: 488, Percentage of Translations Predicted: 156.15%
pronoun - Translations in Apertium Bidix: 170, Percentage of Translations Predicted: 118.82%
preposition - Translations in Apertium Bidix: 711, Percentage of Translations Predicted: 44.16%
